In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import backtrader as bt


In [2]:
import datetime

datetime.datetime(2010,4,5,6,30,20)

datetime.datetime(2010, 4, 5, 6, 30, 20)

In [3]:
import pandas as pd
datapath = os.path.join('./DATA/eurusd_IndicatorDataMinute.csv')
datapath2 = os.path.join('./DATA/gbpusd_IndicatorDataMinute.csv')
datapath = os.path.join('./DATA/SPY_5_min.csv')
df_bt1= pd.read_csv(datapath,  parse_dates=True, index_col=0)


In [4]:
df_bt1.head(10)

,Open,High,Low,Close,Volume
Date,,,,,
2008-05-05 09:45:00,140.88,140.94,140.78,140.94,1832200
2008-05-05 09:50:00,140.93,141.13,140.91,141.11,1394900
2008-05-05 09:55:00,141.11,141.24,141.07,141.20,2073000
2008-05-05 10:00:00,141.20,141.35,141.14,141.18,1634800
2008-05-05 10:05:00,141.19,141.61,141.08,141.08,3928500
2008-05-05 10:10:00,141.09,141.54,141.06,141.54,4572300
2008-05-05 10:15:00,141.54,141.55,141.40,141.40,1943300
2008-05-05 10:20:00,141.40,141.41,141.06,141.12,3430100
2008-05-05 10:25:00,141.11,141.20,141.00,141.02,1448400


In [7]:
import pandas as pd
from rnn_functions import *
from rnn_deploy_functions import *
from tensorflow.keras.models import load_model
from sklearn.externals import joblib
import datetime
import requests

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

# threshold = 0.5
# SEQ_LEN = 60
# model_input = "RNN_Final-040-0.519.model"
# rnn_model = load_model(filepath = model_input, custom_objects=None, compile=False)

# #Load scaler and columns sequence
# scaler = joblib.load("RUNS/EURUSD-60-5-5-5-0.4-0.0002-256-100-1538043840/rnn_scaler.pkl")
# x_columns = joblib.load("RUNS/EURUSD-60-5-5-5-0.4-0.0002-256-100-1538043840/x_columns.pkl")
# df = pd.read_csv("./DATA/x_82_ETF_FOREX_5MIN_RETONLY.csv")
# df["Date"] = pd.to_datetime(df["Date"])
# df = df.set_index("Date")

# datetime_now = datetime.datetime(2008, 10,7,16,40,0)


In [8]:
datetime_now = datetime.datetime(2011,5,21,10,20,15)
# model_input_x = query_x(df, datetime_now, x_columns, SEQ_LEN, scaler)
# predicted_signal = predict_from_model(model_input_x, rnn_model)
# predicted_signal

# df.columns




In [12]:
# Create a Stratey
class TestStrategy(bt.Strategy):
   
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or (self.datas[0].datetime.datetime(0))
        onlog = True
            

        if onlog:
            print('%s, %s' % (dt.strftime('%Y/%m/%d %H:%M:%S'), txt))       

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
#                 self.log(
#                     'BUY EXECUTED, Price: %.4f, Cost: %.4f, Comm %.4f' %
#                     (order.executed.price,
#                      order.executed.value,
#                      order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                ""
#                 self.log('SELL EXECUTED, Price: %.4f, Cost: %.4f, Comm %.4f' %
#                          (order.executed.price,
#                           order.executed.value,
#                           order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

#         self.log('OPERATION PROFIT, GROSS %.4f, NET %.4f' % (trade.pnl, trade.pnlcomm))
#         self.log('Portfolio Value %.4f' % broker.getvalue())
    
    def next(self):
        if self.order:
            return
        
        datetime_now = self.datas[0].datetime.datetime(0)
#             model_input_x = query_x(df, datetime_now, x_columns, SEQ_LEN, scaler)
#             predicted_signal = predict_from_model(model_input_x, rnn_model, threshold)

        url = "http://192.168.1.130:3030/predict_using_datetime?year={year}&month={month}&day={day}&hour={hour}&minute={minute}&second={second}"
        url2 = url.format(year = datetime_now.year, month = datetime_now.month,day = datetime_now.day,
                          hour = datetime_now.hour, minute = datetime_now.minute, second = datetime_now.second)
        r = requests.get(url2)
        signal_raw = float(r.json()["signal_raw"])
        predicted_signal = 0
        self.log("{}, {}".format(signal_raw, predicted_signal))

        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...
           
            
            if signal_raw > 0.5:
#                 self.log('self.datas[0].close[0] = {}'.format(self.datas[0].close[0]))
                self.log('%s BUY CREATE, %.4f' % (self.datas[0].datetime.datetime(0).strftime('%Y/%m/%d %H:%M:%S'), self.dataclose[0]))
                self.order = self.buy(self.datas[0], size = 1)

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 4):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.4f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell(self.datas[0], size = 1)


In [13]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

#     datapath = os.path.join('./DATA/eurusd_IndicatorDataMinute.csv')
    data = bt.feeds.PandasData(dataname=df_bt1,
                                fromdate=datetime.datetime(2012, 1, 1),
                                todate=datetime.datetime(2015, 3, 31),
                               timeframe=bt.TimeFrame.Minutes,
                               compression=5)
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.0001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
#     cerebro.addanalyzer(bt.analyzers.Returns, timeframe=bt.TimeFrame.Minutes, compression=5, _name='mysharpe')
#     cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Minutes, compression=5, _name='mysharpe')
#     cerebro.addanalyzer(bt.analyzers.SQN, _name='mysqn')
#     cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradea')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    
    # Run over everything
    strategies = cerebro.run()
    
    thestrat = strategies[0]
    pyfoliozer = thestrat.analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    
#     print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())
#     print('SQN:', thestrat.analyzers.mysqn.get_analysis())
#     print('Trades:', thestrat.analyzers.tradea.get_analysis())
    
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())



Starting Portfolio Value: 100000.00
2012/01/03 09:35:00, 0.21077919, 0
2012/01/03 09:40:00, 0.22682452, 0
2012/01/03 09:45:00, 0.26557967, 0
2012/01/03 09:50:00, 0.5883064, 0
2012/01/03 09:50:00, 2012/01/03 09:50:00 BUY CREATE, 127.9200
2012/01/03 09:55:00, 0.8161763, 0
2012/01/03 10:00:00, 0.86348283, 0
2012/01/03 10:05:00, 0.8797628, 0
2012/01/03 10:10:00, 0.88452435, 0
2012/01/03 10:15:00, 0.88003945, 0
2012/01/03 10:15:00, SELL CREATE, 128.2700
2012/01/03 10:20:00, 0.875652, 0
2012/01/03 10:20:00, 2012/01/03 10:20:00 BUY CREATE, 128.3000
2012/01/03 10:25:00, 0.871743, 0
2012/01/03 10:30:00, 0.8686136, 0
2012/01/03 10:35:00, 0.86657447, 0
2012/01/03 10:40:00, 0.8658611, 0
2012/01/03 10:45:00, 0.8653611, 0
2012/01/03 10:45:00, SELL CREATE, 128.1700
2012/01/03 10:50:00, 0.86533827, 0
2012/01/03 10:50:00, 2012/01/03 10:50:00 BUY CREATE, 128.1500
2012/01/03 10:55:00, 0.86537975, 0
2012/01/03 11:00:00, 0.8657274, 0
2012/01/03 11:05:00, 0.86574477, 0
2012/01/03 11:10:00, 0.8654864, 0
2012

2012/01/05 09:55:00, 0.6402527, 0
2012/01/05 09:55:00, 2012/01/05 09:55:00 BUY CREATE, 126.8300
2012/01/05 10:00:00, 0.8232763, 0
2012/01/05 10:05:00, 0.8648221, 0
2012/01/05 10:10:00, 0.8785112, 0
2012/01/05 10:15:00, 0.88136166, 0
2012/01/05 10:20:00, 0.87575173, 0
2012/01/05 10:20:00, SELL CREATE, 126.9900
2012/01/05 10:25:00, 0.8696921, 0
2012/01/05 10:25:00, 2012/01/05 10:25:00 BUY CREATE, 127.0100
2012/01/05 10:30:00, 0.86427027, 0
2012/01/05 10:35:00, 0.85973394, 0
2012/01/05 10:40:00, 0.855965, 0
2012/01/05 10:45:00, 0.8517089, 0
2012/01/05 10:50:00, 0.84515107, 0
2012/01/05 10:50:00, SELL CREATE, 126.7700
2012/01/05 10:55:00, 0.8354738, 0
2012/01/05 10:55:00, 2012/01/05 10:55:00 BUY CREATE, 126.7200
2012/01/05 11:00:00, 0.828603, 0
2012/01/05 11:05:00, 0.8331832, 0
2012/01/05 11:10:00, 0.84175223, 0
2012/01/05 11:15:00, 0.84705883, 0
2012/01/05 11:20:00, 0.8503492, 0
2012/01/05 11:20:00, SELL CREATE, 126.8800
2012/01/05 11:25:00, 0.8526068, 0
2012/01/05 11:25:00, 2012/01/05 11

2012/01/09 10:25:00, 0.8855084, 0
2012/01/09 10:30:00, 0.88721293, 0
2012/01/09 10:30:00, SELL CREATE, 128.1300
2012/01/09 10:35:00, 0.8810221, 0
2012/01/09 10:35:00, 2012/01/09 10:35:00 BUY CREATE, 128.0000
2012/01/09 10:40:00, 0.8751608, 0
2012/01/09 10:45:00, 0.8702011, 0
2012/01/09 10:50:00, 0.8663492, 0
2012/01/09 10:55:00, 0.8634204, 0
2012/01/09 11:00:00, 0.86122954, 0
2012/01/09 11:00:00, SELL CREATE, 127.7400
2012/01/09 11:05:00, 0.859583, 0
2012/01/09 11:05:00, 2012/01/09 11:05:00 BUY CREATE, 127.7400
2012/01/09 11:10:00, 0.85808605, 0
2012/01/09 11:15:00, 0.8560441, 0
2012/01/09 11:20:00, 0.85318327, 0
2012/01/09 11:25:00, 0.84998345, 0
2012/01/09 11:30:00, 0.8468339, 0
2012/01/09 11:30:00, SELL CREATE, 127.6000
2012/01/09 11:35:00, 0.84431094, 0
2012/01/09 11:35:00, 2012/01/09 11:35:00 BUY CREATE, 127.5900
2012/01/09 11:40:00, 0.84448457, 0
2012/01/09 11:45:00, 0.8455004, 0
2012/01/09 11:50:00, 0.8459915, 0
2012/01/09 11:55:00, 0.8459434, 0
2012/01/09 12:00:00, 0.8451917, 0

2012/01/11 10:55:00, 0.85671455, 0
2012/01/11 10:55:00, 2012/01/11 10:55:00 BUY CREATE, 128.9400
2012/01/11 11:00:00, 0.8560356, 0
2012/01/11 11:05:00, 0.8548243, 0
2012/01/11 11:10:00, 0.8534811, 0
2012/01/11 11:15:00, 0.8525897, 0
2012/01/11 11:20:00, 0.8534009, 0
2012/01/11 11:20:00, SELL CREATE, 128.8000
2012/01/11 11:25:00, 0.8545392, 0
2012/01/11 11:25:00, 2012/01/11 11:25:00 BUY CREATE, 128.8100
2012/01/11 11:30:00, 0.8554849, 0
2012/01/11 11:35:00, 0.8563709, 0
2012/01/11 11:40:00, 0.85687375, 0
2012/01/11 11:45:00, 0.8568467, 0
2012/01/11 11:50:00, 0.85697544, 0
2012/01/11 11:50:00, SELL CREATE, 128.9700
2012/01/11 11:55:00, 0.85651356, 0
2012/01/11 11:55:00, 2012/01/11 11:55:00 BUY CREATE, 128.9000
2012/01/11 12:00:00, 0.85629165, 0
2012/01/11 12:05:00, 0.8562953, 0
2012/01/11 12:10:00, 0.85656583, 0
2012/01/11 12:15:00, 0.85668033, 0
2012/01/11 12:20:00, 0.85614675, 0
2012/01/11 12:20:00, SELL CREATE, 128.9800
2012/01/11 12:25:00, 0.8548344, 0
2012/01/11 12:25:00, 2012/01/11

2012/01/13 11:05:00, 0.8565952, 0
2012/01/13 11:10:00, 0.85566485, 0
2012/01/13 11:15:00, 0.8547967, 0
2012/01/13 11:20:00, 0.85397816, 0
2012/01/13 11:20:00, SELL CREATE, 128.4800
2012/01/13 11:25:00, 0.8534905, 0
2012/01/13 11:25:00, 2012/01/13 11:25:00 BUY CREATE, 128.5500
2012/01/13 11:30:00, 0.852365, 0
2012/01/13 11:35:00, 0.85138226, 0
2012/01/13 11:40:00, 0.8501629, 0
2012/01/13 11:45:00, 0.8488249, 0
2012/01/13 11:50:00, 0.8473131, 0
2012/01/13 11:50:00, SELL CREATE, 128.5900
2012/01/13 11:55:00, 0.84550416, 0
2012/01/13 11:55:00, 2012/01/13 11:55:00 BUY CREATE, 128.6000
2012/01/13 12:00:00, 0.8425132, 0
2012/01/13 12:05:00, 0.8372499, 0
2012/01/13 12:10:00, 0.8288692, 0
2012/01/13 12:15:00, 0.81747174, 0
2012/01/13 12:20:00, 0.8066379, 0
2012/01/13 12:20:00, SELL CREATE, 128.3500
2012/01/13 12:25:00, 0.79834706, 0
2012/01/13 12:25:00, 2012/01/13 12:25:00 BUY CREATE, 128.3400
2012/01/13 12:30:00, 0.7915074, 0
2012/01/13 12:35:00, 0.7818252, 0
2012/01/13 12:40:00, 0.7617133, 0


...

2012/03/27 12:30:00, 0.8203625, 0
2012/03/27 12:30:00, 2012/03/27 12:30:00 BUY CREATE, 141.6000
2012/03/27 12:35:00, 0.7623551, 0
2012/03/27 12:40:00, 0.65388286, 0
2012/03/27 12:45:00, 0.50832325, 0
2012/03/27 12:50:00, 0.39098293, 0
2012/03/27 12:55:00, 0.34704435, 0
2012/03/27 12:55:00, SELL CREATE, 141.5700
2012/03/27 13:00:00, 0.39925423, 0
2012/03/27 13:05:00, 0.5110625, 0
2012/03/27 13:05:00, 2012/03/27 13:05:00 BUY CREATE, 141.6200
2012/03/27 13:10:00, 0.597008, 0
2012/03/27 13:15:00, 0.6097147, 0
2012/03/27 13:20:00, 0.5772665, 0
2012/03/27 13:25:00, 0.5434348, 0
2012/03/27 13:30:00, 0.56688416, 0
2012/03/27 13:30:00, SELL CREATE, 141.5200
2012/03/27 13:35:00, 0.6466228, 0
2012/03/27 13:35:00, 2012/03/27 13:35:00 BUY CREATE, 141.5000
2012/03/27 13:40:00, 0.7266085, 0
2012/03/27 13:45:00, 0.7634088, 0
2012/03/27 13:50:00, 0.76969266, 0
2012/03/27 13:55:00, 0.75805795, 0
2012/03/27 14:00:00, 0.7263868, 0
2012/03/27 14:00:00, SELL CREATE, 141.6000
2012/03/27 14:05:00, 0.654992, 0

2012/03/29 13:05:00, 0.85218555, 0
2012/03/29 13:10:00, 0.8736611, 0
2012/03/29 13:15:00, 0.8787454, 0
2012/03/29 13:20:00, 0.87813234, 0
2012/03/29 13:20:00, SELL CREATE, 139.3500
2012/03/29 13:25:00, 0.8763332, 0
2012/03/29 13:25:00, 2012/03/29 13:25:00 BUY CREATE, 139.3900
2012/03/29 13:30:00, 0.8743343, 0
2012/03/29 13:35:00, 0.87255764, 0
2012/03/29 13:40:00, 0.8708496, 0
2012/03/29 13:45:00, 0.86946356, 0
2012/03/29 13:50:00, 0.86822474, 0
2012/03/29 13:50:00, SELL CREATE, 139.3600
2012/03/29 13:55:00, 0.8674184, 0
2012/03/29 13:55:00, 2012/03/29 13:55:00 BUY CREATE, 139.3800
2012/03/29 14:00:00, 0.8664976, 0
2012/03/29 14:05:00, 0.8660271, 0
2012/03/29 14:10:00, 0.86477375, 0
2012/03/29 14:15:00, 0.8637999, 0
2012/03/29 14:20:00, 0.8625732, 0
2012/03/29 14:20:00, SELL CREATE, 139.7200
2012/03/29 14:25:00, 0.8607751, 0
2012/03/29 14:25:00, 2012/03/29 14:25:00 BUY CREATE, 139.6800
2012/03/29 14:30:00, 0.8589863, 0
2012/03/29 14:35:00, 0.85698056, 0
2012/03/29 14:40:00, 0.8549872, 

2012/04/02 13:30:00, 0.20731805, 0
2012/04/02 13:35:00, 0.20924303, 0
2012/04/02 13:40:00, 0.21376143, 0
2012/04/02 13:45:00, 0.21952209, 0
2012/04/02 13:50:00, 0.22719051, 0
2012/04/02 13:55:00, 0.23811279, 0
2012/04/02 14:00:00, 0.30162585, 0
2012/04/02 14:05:00, 0.5094099, 0
2012/04/02 14:05:00, 2012/04/02 14:05:00 BUY CREATE, 142.0300
2012/04/02 14:10:00, 0.6620427, 0
2012/04/02 14:15:00, 0.7285416, 0
2012/04/02 14:20:00, 0.72955924, 0
2012/04/02 14:25:00, 0.69087076, 0
2012/04/02 14:30:00, 0.64076155, 0
2012/04/02 14:30:00, SELL CREATE, 142.0200
2012/04/02 14:35:00, 0.62215847, 0
2012/04/02 14:35:00, 2012/04/02 14:35:00 BUY CREATE, 142.0600
2012/04/02 14:40:00, 0.6714523, 0
2012/04/02 14:45:00, 0.7436166, 0
2012/04/02 14:50:00, 0.80631083, 0
2012/04/02 14:55:00, 0.83937645, 0
2012/04/02 15:00:00, 0.8515398, 0
2012/04/02 15:00:00, SELL CREATE, 142.1800
2012/04/02 15:05:00, 0.8547023, 0
2012/04/02 15:05:00, 2012/04/02 15:05:00 BUY CREATE, 142.1500
2012/04/02 15:10:00, 0.8515127, 0
2

2012/04/04 14:25:00, 0.8552616, 0
2012/04/04 14:30:00, 0.852235, 0
2012/04/04 14:30:00, SELL CREATE, 139.6300
2012/04/04 14:35:00, 0.8513494, 0
2012/04/04 14:35:00, 2012/04/04 14:35:00 BUY CREATE, 139.7700
2012/04/04 14:40:00, 0.8530145, 0
2012/04/04 14:45:00, 0.8540286, 0
2012/04/04 14:50:00, 0.85426885, 0
2012/04/04 14:55:00, 0.8540221, 0
2012/04/04 15:00:00, 0.8538131, 0
2012/04/04 15:00:00, SELL CREATE, 139.9000
2012/04/04 15:05:00, 0.85314846, 0
2012/04/04 15:05:00, 2012/04/04 15:05:00 BUY CREATE, 139.8700
2012/04/04 15:10:00, 0.8524672, 0
2012/04/04 15:15:00, 0.8514431, 0
2012/04/04 15:20:00, 0.84975517, 0
2012/04/04 15:25:00, 0.8464191, 0
2012/04/04 15:30:00, 0.8396126, 0
2012/04/04 15:30:00, SELL CREATE, 139.9600
2012/04/04 15:35:00, 0.83320725, 0
2012/04/04 15:35:00, 2012/04/04 15:35:00 BUY CREATE, 140.0900
2012/04/04 15:40:00, 0.8294838, 0
2012/04/04 15:45:00, 0.82741565, 0
2012/04/04 15:50:00, 0.82700044, 0
2012/04/04 15:55:00, 0.8256072, 0
2012/04/04 16:00:00, 0.82149434, 0

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
cerebro.plot()

In [ ]:
import pyfolio as pf
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions)

In [14]:
np.cumprod(1+returns).plot()

NameError: name 'returns' is not defined

In [ ]:
aa = returns.reset_index()["return"].values

In [ ]:
rr = returns.reset_index()

In [ ]:
rr["index"] = pd.to_datetime(rr["index"])

In [ ]:
import matplotlib.pyplot as plt
rr.set_index("index").plot()
plt.show()

In [ ]:
np.mean(rr["return"] > 0)
